In [33]:
# Original Dataset: https://www.cs.toronto.edu/~kriz/cifar.html for more information
# Load of necessary libraries
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import Conv2D
from tensorflow.keras.optimizers import Adam
from keras.layers.pooling import MaxPooling2D
from tensorflow.keras.utils import to_categorical
# to make the example replicable
np.random.seed(42)
# Load of the dataset
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [34]:
X_train.shape, Y_train.shape

((50000, 32, 32, 3), (50000, 1))

### reduce dataset size

In [35]:
df = pd.DataFrame(list(zip(X_train, Y_train)), columns =['Image', 'label']) 
val = df.sample(frac=0.2)
X_train = np.array([ i for i in list(val['Image'])])
Y_train = np.array([ [i[0]] for i in list(val['label'])])

In [36]:
X_train.shape, Y_train.shape

((10000, 32, 32, 3), (10000, 1))

## Normalization

In [37]:
# Centering the data
X_train_mean = np.mean(X_train, axis = 0)
X_train_cent = X_train - X_train_mean
# Normalization
X_train_std = np.std(X_train, axis = 0)
X_train_norm = X_train_cent / X_train_std

## Train/Test

In [38]:
# preparing test set
X_test_norm = (X_test - X_train_mean) / X_train_std

## Conv Model

In [39]:
from keras.layers import BatchNormalization, Activation
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm


class Convolutional:
    
    def __init__(self, desc, batch_size=128, shuffle=True, epochs=10, batch_norm=False, 
                 k_regularizer=None, k_constraint=None, dropout=False, 
                 dropout_val=0.5, max_pooling=False, strides=1):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.epochs = epochs
        self.desc = desc
        self.scores = []
        
        if max_pooling and dropout:
            print('why')
            return None
        
        # Inizializting the model
        self.model = Sequential()
        # Defining a convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(32, 32, 3)))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # Defining a second convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None,
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # Defining a third convolutional layer
        self.model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
        if batch_norm: self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        if max_pooling: self.model.add(MaxPooling2D(
            pool_size=(2, 2), strides=strides, padding="valid", data_format=None,
        ))
        if dropout: self.model.add(Dropout(dropout_val/2))

        # We include our classifier
        self.model.add(Flatten())
        self.model.add(Dense(1024, activation='relu', kernel_regularizer=k_regularizer, kernel_constraint=k_constraint))
        if dropout: self.model.add(Dropout(dropout_val))
        self.model.add(Dense(10, activation='softmax'))
        
        # Compiling the model
        self.model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(lr=0.0001, decay=1e-6),
                      metrics=['accuracy'])
        
    def train(self, X_train, Y_train, X_test, Y_test):
        # Training of the model
        self.model.fit(X_train, to_categorical(Y_train),
                  batch_size=self.batch_size,
                  shuffle=self.shuffle,
                  epochs=self.epochs,
                  validation_data=(X_test, to_categorical(Y_test)))
        
        # Evaluation of the model
        self.scores = self.model.evaluate(X_test, to_categorical(Y_test))
        return self.scores
    
    def __str__(self):
        return self.desc+":\t Loss: "+self.scores[0]+"\t Accuracy: "+self.scores[1]
        
'''
print('Loss: %.3f' % scores[0])
print('Accuracy: %.3f' % scores[1])
'''

"\nprint('Loss: %.3f' % scores[0])\nprint('Accuracy: %.3f' % scores[1])\n"

## Instantiate models & fit

In [40]:
from keras.layers import BatchNormalization, Activation
from keras.regularizers import l1, l2, l1_l2
from keras.constraints import max_norm
from keras.layers import Dropout


models = []

batch_size = 512
epochs = 10

## Batch Norm
models.append(Convolutional(desc="Batch_Norm", batch_size=batch_size, epochs=epochs, \
                             batch_norm=True))

## Regularization
'''
for i in range(-3, 0):
    models.append(Convolutional(desc="Regularization_l1_10^"+str(i), \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1(10**i)))
    models.append(Convolutional(desc="Regularization_l2_10^"+str(i), \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l2(10**i)))
    models.append(Convolutional(desc="Regularization_elastic_10^"+str(i), \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1_l2(10**i)))
'''
models.append(Convolutional(desc="Regularization_l1_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1(10**-2)))
models.append(Convolutional(desc="Regularization_l2_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l2(10**-2)))
models.append(Convolutional(desc="Regularization_elastic_10^-2", \
                            batch_size=batch_size, epochs=epochs, \
                            k_regularizer=l1_l2(10**-2)))
    
## Max Norm & Dropout
models.append(Convolutional(desc="Max_Norm", \
                            batch_size=batch_size, epochs=epochs, \
                            k_constraint=max_norm(3. )))

models.append(Convolutional(desc="Dropout", \
                            batch_size=batch_size, epochs=epochs, \
                            dropout = True))

models.append(Convolutional(desc="Max_Norm_And_Dropout", \
                            batch_size=batch_size, epochs=epochs, \
                            k_constraint=max_norm(3. ), dropout=True))

## Max Pooling
models.append(Convolutional(desc="Max_Pooling", \
                            batch_size=batch_size, epochs=epochs, \
                            max_pooling=True))




'''

from keras.regularizers import l1, l2, l1_l2
k_regs = []

k_regs.extend([l1(10**i) for i in range(-3,0)])
k_regs.extend([l2(10**i) for i in range(-3,0)])
k_regs.extend([l1_l2(10**i) for i in range(-3,0)])

from keras.constraints import max_norm
k_cons = []
k_cons.extend([max_norm(i) for i in range(3,5)])
'''

'\n\nfrom keras.regularizers import l1, l2, l1_l2\nk_regs = []\n\nk_regs.extend([l1(10**i) for i in range(-3,0)])\nk_regs.extend([l2(10**i) for i in range(-3,0)])\nk_regs.extend([l1_l2(10**i) for i in range(-3,0)])\n\nfrom keras.constraints import max_norm\nk_cons = []\nk_cons.extend([max_norm(i) for i in range(3,5)])\n'

## Fit models & Report scores

In [41]:
for m in models:
    m.train(X_train_norm, Y_train, X_test_norm, Y_test)
    print(m)

Epoch 1/10
20/20 [==============================] - 168s 8s/step - loss: 6.9172 - accuracy: 0.1866 - val_loss: 2.2860 - val_accuracy: 0.2090
Epoch 2/10
20/20 [==============================] - 161s 8s/step - loss: 1.8438 - accuracy: 0.3668 - val_loss: 2.2782 - val_accuracy: 0.2640
Epoch 3/10
20/20 [==============================] - 173s 9s/step - loss: 1.4096 - accuracy: 0.5249 - val_loss: 2.2777 - val_accuracy: 0.2536
Epoch 4/10
 1/20 [>.............................] - ETA: 2:19 - loss: 1.1417 - accuracy: 0.6562

KeyboardInterrupt: 